## This notebook is run on Google Colab originally

In [ ]:
import pandas as pd
import time
import numpy as np
import datetime
import openpyxl
import pandas as pd
import pytz
import urllib
import io
import json
import requests

In [ ]:
!pip install beautifulsoup4
import bs4

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# get data from HK gov
response1 = requests.get('http://www.chp.gov.hk/files/misc/building_list_chi.csv')

file_object1 = io.StringIO(response1.content.decode('utf-8'))
df_building_list = pd.read_csv(file_object1)

In [ ]:
df_building_list.head()

,地區,大廈名單,最後個案居住日期,相關疑似/確診個案
0,觀塘,鯉魚門邨鯉隆樓,NaN,8483
1,屯門,山景邨景安樓,NaN,8484
2,深水埗,李鄭屋邨和平樓,NaN,8485
3,大埔,寶雅苑興和閣,NaN,8488
4,西貢,富康花園2座,NaN,8489


In [ ]:
df_building_list.tail()

,地區,大廈名單,最後個案居住日期,相關疑似/確診個案
1008,西貢,西貢將軍澳政府綜合大樓 (非住宅),07/01/2021,9136
1009,沙田,美高工業大廈 (非住宅),07/01/2021,9199
1010,荃灣,圓玄學院 (非住宅),07/01/2021,9201
1011,葵青,青衣救護站 (非住宅),07/01/2021,9206
1012,油尖旺,聯合廣場 好彩海鮮酒家 (非住宅),08/01/2021,9203


In [ ]:
# get data from HK gov
response2 = requests.get('http://www.chp.gov.hk/files/misc/enhanced_sur_covid_19_chi.csv')

file_object2 = io.StringIO(response2.content.decode('utf-8'))
df_case = pd.read_csv(file_object2)

In [ ]:
df_case.dtypes

個案編號          int64
報告日期         object
發病日期         object
性別           object
年齡            int64
入住醫院名稱      float64
住院/出院/死亡     object
香港/非香港居民     object
個案分類*        object
確診/疑似個案      object
dtype: object

In [ ]:
df_case.tail()

,個案編號,報告日期,發病日期,性別,年齡,入住醫院名稱,住院/出院/死亡,香港/非香港居民,個案分類*,確診/疑似個案
9207,9208,09/01/2021,無病徵,女,58,NaN,有待提供,香港居民,與本地個案有流行病學關連,確診
9208,9209,09/01/2021,無病徵,男,29,NaN,有待提供,香港居民,輸入個案,確診
9209,9210,09/01/2021,08/01/2021,男,31,NaN,有待提供,香港居民,與本地個案有流行病學關連,確診
9210,9211,09/01/2021,05/01/2021,男,66,NaN,有待提供,香港居民,本地個案,確診
9211,9212,09/01/2021,07/01/2021,女,61,NaN,有待提供,香港居民,本地個案,確診


In [ ]:
df_building_list_modified = pd.DataFrame(columns = df_building_list.columns)

In [ ]:
for i, row in enumerate(df_building_list.itertuples()):
  
  IDs = row._4.split(', ')
  if len(IDs) == 1:
    df_building_list_modified = df_building_list_modified.append(pd.DataFrame([[row.地區, row.大廈名單, row.最後個案居住日期, int(row._4)]], columns=df_building_list_modified.columns))
  elif len(IDs)>1:
    #print(row)
    #print(f"{i} {row._4} {len(IDs)}")
    for id in IDs: 
      df_building_list_modified = df_building_list_modified.append(pd.DataFrame([[row.地區, row.大廈名單, row.最後個案居住日期, int(id)]], columns=df_building_list_modified.columns))
    

In [ ]:
df_building_list_modified.head()

,地區,大廈名單,最後個案居住日期,相關疑似/確診個案
0,觀塘,鯉魚門邨鯉隆樓,NaN,8483
0,屯門,山景邨景安樓,NaN,8484
0,深水埗,李鄭屋邨和平樓,NaN,8485
0,大埔,寶雅苑興和閣,NaN,8488
0,西貢,富康花園2座,NaN,8489


In [ ]:
df_building_list_modified.tail()

,地區,大廈名單,最後個案居住日期,相關疑似/確診個案
0,西貢,西貢將軍澳政府綜合大樓 (非住宅),07/01/2021,9136
0,沙田,美高工業大廈 (非住宅),07/01/2021,9199
0,荃灣,圓玄學院 (非住宅),07/01/2021,9201
0,葵青,青衣救護站 (非住宅),07/01/2021,9206
0,油尖旺,聯合廣場 好彩海鮮酒家 (非住宅),08/01/2021,9203


Modifiying 大廈名單

In [ ]:
df_building_list_modified[['地址','地址性質']] = df_building_list_modified['大廈名單'].str.split(f' \\(',expand=True)

In [ ]:
df_building_list_modified.loc[df_building_list_modified['地址性質']=='非住宅)','地址性質'] = '非住宅'

In [ ]:
df_building_list_modified.loc[df_building_list_modified['地址性質']!='非住宅','地址性質'] = '住宅'

In [ ]:
df_building_list_modified['地區地址'] = df_building_list_modified['地區'] + df_building_list_modified['地址']

In [ ]:
df_building_list_modified.reset_index(drop=True, inplace=True)
df_building_list_modified

,地區,大廈名單,最後個案居住日期,相關疑似/確診個案,地址,地址性質,地區地址
0,觀塘,鯉魚門邨鯉隆樓,NaN,8483,鯉魚門邨鯉隆樓,住宅,觀塘鯉魚門邨鯉隆樓
1,屯門,山景邨景安樓,NaN,8484,山景邨景安樓,住宅,屯門山景邨景安樓
2,深水埗,李鄭屋邨和平樓,NaN,8485,李鄭屋邨和平樓,住宅,深水埗李鄭屋邨和平樓
3,大埔,寶雅苑興和閣,NaN,8488,寶雅苑興和閣,住宅,大埔寶雅苑興和閣
4,西貢,富康花園2座,NaN,8489,富康花園2座,住宅,西貢富康花園2座
...,...,...,...,...,...,...,...
1352,西貢,西貢將軍澳政府綜合大樓 (非住宅),07/01/2021,9136,西貢將軍澳政府綜合大樓,非住宅,西貢西貢將軍澳政府綜合大樓
1353,沙田,美高工業大廈 (非住宅),07/01/2021,9199,美高工業大廈,非住宅,沙田美高工業大廈
1354,荃灣,圓玄學院 (非住宅),07/01/2021,9201,圓玄學院,非住宅,荃灣圓玄學院
1355,葵青,青衣救護站 (非住宅),07/01/2021,9206,青衣救護站,非住宅,葵青青衣救護站


Find Latitude, Longtitude of locations from Google Map API

In [ ]:
# For example
location="灣仔名珠城"

In [ ]:
API_KEY = "YOUR_KEY"

In [ ]:
params = {
    'key': API_KEY,
    'address': location
}
base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'

In [ ]:
response = requests.get(base_url, params=params).json()
response.keys()

dict_keys(['results', 'status'])

In [ ]:
if response['status']=='OK':
  geometry = response['results'][0]['geometry']
  lat = geometry['location']['lat']
  lng = geometry['location']['lng']
  print(lat,lng)

22.2810532 114.1858291


In [ ]:
# use Google Map API to find latitude, longitude
def query_location(location, API_KEY=API_KEY):
  params = {
    'key': API_KEY,
    'address': location
  }
  base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
  response = requests.get(base_url, params=params).json()
  if response['status']=='OK':
    geometry = response['results'][0]['geometry']
    lat = geometry['location']['lat']
    lng = geometry['location']['lng']
    return lat, lng
  else:
    return np.nan, np.nan

In [ ]:
geo_info = query_location(location)
geo_info

(22.2810532, 114.1858291)

In [ ]:
for i, row in enumerate(df_building_list_modified.itertuples()):
  lat, lon = query_location(row.地區地址)
  df_building_list_modified.loc[i,['Latitude']], df_building_list_modified.loc[i,['Longitude']] = lat, lon
  print(i,row.地區地址, lat, lon)


0 觀塘鯉魚門邨鯉隆樓 22.2933512 114.2396535
1 屯門山景邨景安樓 22.3981329 113.9662423
2 深水埗李鄭屋邨和平樓 22.3389326 114.1585661
3 大埔寶雅苑興和閣 22.4506961 114.1587519
4 西貢富康花園2座 22.3106467 114.261316
5 觀塘德福花園H1座 22.3237862 114.2129936
6 荃灣象山邨秀山樓 22.378753 114.13157
7 南區如心南灣海景酒店 22.2497896 114.1681339
8 西貢安寧花園5座 22.3160735 114.2624072
9 黃大仙黃大仙下邨龍盛樓 22.3402708 114.1925359
10 九龍城天鑄5座 22.3131109 114.1815051
11 九龍城天鑄5座 22.3131109 114.1815051
12 屯門欣田邨喜田樓 22.4152562 113.9758875
13 屯門欣田邨喜田樓 22.4152562 113.9758875
14 九龍城寶峰大廈 22.3309857 114.1879624
15 葵青美景花園7座 22.3445061 114.1075422
16 西貢日出康城2A期領都5座Water Lilies 22.2973028 114.2704636
17 觀塘平田邨平善樓 22.3072304 114.2362171
18 黃大仙安強大樓 22.3342193 114.1965806
19 黃大仙彩雲邨紫霄樓 22.333161 114.213405
20 黃大仙彩雲邨紫霄樓 22.333161 114.213405
21 黃大仙采頤花園11座 22.336503 114.2021304
22 荃灣象山邨秀山樓 22.378753 114.13157
23 東區明華大廈A座 22.2793699 114.2312389
24 九龍城美景樓1期 22.3161111 114.1908333
25 中西區華美達海景酒店 22.2869928 114.1435356
26 北區安圃村 22.490185 114.117131
27 觀塘宜安中心 22.3152556 114.2233109
28 黃大仙慈正邨正康樓 22.34918

In [ ]:
df_building_list_modified.tail()

,地區,大廈名單,最後個案居住日期,相關疑似/確診個案,地址,地址性質,地區地址,Latitude,Longitude
1352,西貢,西貢將軍澳政府綜合大樓 (非住宅),07/01/2021,9136,西貢將軍澳政府綜合大樓,非住宅,西貢西貢將軍澳政府綜合大樓,22.317253,114.268165
1353,沙田,美高工業大廈 (非住宅),07/01/2021,9199,美高工業大廈,非住宅,沙田美高工業大廈,22.398665,114.193306
1354,荃灣,圓玄學院 (非住宅),07/01/2021,9201,圓玄學院,非住宅,荃灣圓玄學院,22.383519,114.123011
1355,葵青,青衣救護站 (非住宅),07/01/2021,9206,青衣救護站,非住宅,葵青青衣救護站,22.350581,114.106528
1356,油尖旺,聯合廣場 好彩海鮮酒家 (非住宅),08/01/2021,9203,聯合廣場 好彩海鮮酒家,非住宅,油尖旺聯合廣場 好彩海鮮酒家,22.323526,114.168775


In [ ]:
#pd.io.formats.excel.header_style = None

In [ ]:
now = datetime.datetime.now()
# assuming now contains a timezone aware datetime
tz = pytz.timezone('Asia/Hong_Kong')
time_now = now.astimezone(tz).strftime('%Y-%m-%d_%H%M%S')
print(time_now)

2021-01-10_203814


In [ ]:
df_building_list_modified.to_excel(f"gdrive/My Drive/covid HK/df_building_list_modified_{time_now}.xlsx", index=False)

In [ ]:
df_combined = df_case.merge(df_building_list_modified,\
               left_on='個案編號', right_on='相關疑似/確診個案', how='left')

Modifying date columns

In [ ]:
df_combined['報告日期'] = pd.to_datetime(df_combined['報告日期'], format='%d/%m/%Y').dt.date
df_combined['最後個案居住日期'] = pd.to_datetime(df_combined['最後個案居住日期'], format='%d/%m/%Y').dt.date

In [ ]:
df_combined.tail()

,個案編號,報告日期,發病日期,性別,年齡,入住醫院名稱,住院/出院/死亡,香港/非香港居民,個案分類*,確診/疑似個案,地區,大廈名單,最後個案居住日期,相關疑似/確診個案,地址,地址性質,地區地址,Latitude,Longitude
9663,9208,2021-01-09,無病徵,女,58,NaN,有待提供,香港居民,與本地個案有流行病學關連,確診,屯門,山景邨景業樓,NaT,9208,山景邨景業樓,住宅,屯門山景邨景業樓,22.398700,113.966268
9664,9209,2021-01-09,無病徵,男,29,NaN,有待提供,香港居民,輸入個案,確診,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
9665,9210,2021-01-09,08/01/2021,男,31,NaN,有待提供,香港居民,與本地個案有流行病學關連,確診,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
9666,9211,2021-01-09,05/01/2021,男,66,NaN,有待提供,香港居民,本地個案,確診,九龍城,美利大廈,NaT,9211,美利大廈,住宅,九龍城美利大廈,22.317843,114.189889
9667,9212,2021-01-09,07/01/2021,女,61,NaN,有待提供,香港居民,本地個案,確診,東區,興華(一)邨興翠樓,NaT,9212,興華(一)邨興翠樓,住宅,東區興華(一)邨興翠樓,22.262735,114.234824


In [ ]:
def return_date(x):
  try:
    report_time = datetime.datetime.strptime(x, '%d/%m/%Y').date()
    return report_time, "沒有"
  except Exception:
    return np.nan, x

In [ ]:
df_combined[['發病日期', '發病描述']] = df_combined.apply(lambda row: return_date(row['發病日期']), axis=1, result_type="expand")

In [ ]:
df_combined

,個案編號,報告日期,發病日期,性別,年齡,入住醫院名稱,住院/出院/死亡,香港/非香港居民,個案分類*,確診/疑似個案,地區,大廈名單,最後個案居住日期,相關疑似/確診個案,地址,地址性質,地區地址,Latitude,Longitude,發病描述
0,1,2020-01-23,2020-01-21,男,39,NaN,出院,非香港居民,輸入個案,確診,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,沒有
1,2,2020-01-23,2020-01-18,男,56,NaN,出院,香港居民,輸入個案,確診,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,沒有
2,3,2020-01-24,2020-01-20,女,62,NaN,出院,非香港居民,輸入個案,確診,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,沒有
3,4,2020-01-24,2020-01-23,女,62,NaN,出院,非香港居民,輸入個案,確診,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,沒有
4,5,2020-01-24,2020-01-23,男,63,NaN,出院,非香港居民,輸入個案,確診,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,沒有
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9663,9208,2021-01-09,NaN,女,58,NaN,有待提供,香港居民,與本地個案有流行病學關連,確診,屯門,山景邨景業樓,NaT,9208,山景邨景業樓,住宅,屯門山景邨景業樓,22.398700,113.966268,無病徵
9664,9209,2021-01-09,NaN,男,29,NaN,有待提供,香港居民,輸入個案,確診,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,無病徵
9665,9210,2021-01-09,2021-01-08,男,31,NaN,有待提供,香港居民,與本地個案有流行病學關連,確診,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,沒有
9666,9211,2021-01-09,2021-01-05,男,66,NaN,有待提供,香港居民,本地個案,確診,九龍城,美利大廈,NaT,9211,美利大廈,住宅,九龍城美利大廈,22.317843,114.189889,沒有


In [ ]:
df_combined.dtypes

個案編號          object
報告日期          object
發病日期          object
性別            object
年齡             int64
入住醫院名稱       float64
住院/出院/死亡      object
香港/非香港居民      object
個案分類*         object
確診/疑似個案       object
地區            object
大廈名單          object
最後個案居住日期      object
相關疑似/確診個案     object
地址            object
地址性質          object
地區地址          object
Latitude     float64
Longitude    float64
發病描述          object
dtype: object

In [ ]:
df_combined

,個案編號,報告日期,發病日期,性別,年齡,入住醫院名稱,住院/出院/死亡,香港/非香港居民,個案分類*,確診/疑似個案,地區,大廈名單,最後個案居住日期,相關疑似/確診個案,地址,地址性質,地區地址,Latitude,Longitude,發病描述
0,1,2020-01-23,2020-01-21,男,39,NaN,出院,非香港居民,輸入個案,確診,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,沒有
1,2,2020-01-23,2020-01-18,男,56,NaN,出院,香港居民,輸入個案,確診,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,沒有
2,3,2020-01-24,2020-01-20,女,62,NaN,出院,非香港居民,輸入個案,確診,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,沒有
3,4,2020-01-24,2020-01-23,女,62,NaN,出院,非香港居民,輸入個案,確診,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,沒有
4,5,2020-01-24,2020-01-23,男,63,NaN,出院,非香港居民,輸入個案,確診,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,沒有
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9663,9208,2021-01-09,NaN,女,58,NaN,有待提供,香港居民,與本地個案有流行病學關連,確診,屯門,山景邨景業樓,NaT,9208,山景邨景業樓,住宅,屯門山景邨景業樓,22.398700,113.966268,無病徵
9664,9209,2021-01-09,NaN,男,29,NaN,有待提供,香港居民,輸入個案,確診,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,無病徵
9665,9210,2021-01-09,2021-01-08,男,31,NaN,有待提供,香港居民,與本地個案有流行病學關連,確診,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,沒有
9666,9211,2021-01-09,2021-01-05,男,66,NaN,有待提供,香港居民,本地個案,確診,九龍城,美利大廈,NaT,9211,美利大廈,住宅,九龍城美利大廈,22.317843,114.189889,沒有


In [ ]:
regions = set(df_combined.地區)

In [ ]:
regions = [str(r) for r in regions if str(r) != 'nan']

In [ ]:
regions

['屯門',
 '沙田',
 '西貢',
 '黃大仙',
 '中西區',
 '元朗',
 '葵青',
 '離島',
 '九龍城',
 '灣仔',
 '北區',
 '觀塘',
 '大埔',
 '油尖旺',
 '南區',
 '東區',
 '深水埗',
 '荃灣']

In [ ]:
df_combined.to_excel(f"gdrive/My Drive/covid HK/df_combined_{time_now}.xlsx", sheet_name='combined', index=False)

In [ ]:
with pd.ExcelWriter(f"gdrive/My Drive/covid HK/df_combined_{time_now}.xlsx", engine='openpyxl', mode='a') as writer:     
  for r in regions:
      df_region = df_combined.loc[(df_combined.地區 == r)]
      df_region.to_excel(writer, sheet_name=r, index=False)
      writer.save()